# Checking docker setup

## loading needed packages

In [1]:
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.utils import np_utils
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split, KFold
import sklearn.metrics as metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

Using TensorFlow backend.


In [ ]:
# start auto-reloading certain libraries marked with %aimport
%load_ext autoreload
%autoreload 1
# add new source file to auto-reload
%aimport ..src._funcs_
# add function from auto-reloading module
from src._funcs_ import func

## Testing out psycopg2

In [1]:
from psycopg2 import connect

In [2]:
conn = connect(user='postgres', host='pg_serv')

In [3]:
conn_train = connect(dbname='train2017', user='postgres', host='pg_serv')

In [4]:
conn_val = connect(dbname='val2017', user='postgres', host='pg_serv')

In [6]:
import argparse

## Loading up PostgreSQL

In [3]:
import sys; sys.path.append('../src')
from data_pipeline import BuildDatabase, QueryDatabase

In [4]:
train_args = {'dataset':'train2017', 'host':'pg_serv', 'user':'postgres', 'data_dir':'../data/coco/'}

In [4]:
build_trainset = BuildDatabase(**train_args)

Connecting to PostgreSQL server....
	Connected.


In [ ]:
build_trainset.create_tables('../src/coco_dataset.sql')

In [5]:
build_trainset.build_sql()

Building Database...
Loading COCO dataset train2017 information....
Loading annotations into database...
	annotations COPIED.
	COMMITED.
Loading images into database...
	images COPIED.
	COMMITED.
Loading categories into database...
	categories COPIED.
	COMMITED.
Loading licenses into database...
	licenses COPIED.
	COMMITED.
Finished building database.


In [5]:
val_args = {'dataset':'val2017', 'host':'pg_serv', 'user':'postgres', 'data_dir':'../data/coco/'}

In [6]:
build_valset = BuildDatabase(**val_args)
build_valset.create_tables('../src/coco_dataset.sql')
build_valset.build_sql()

Connecting to PostgreSQL server....
	Connected.
Running ../src/coco_dataset.sql...
Executing SQL...
	Done.
	Commited.
Building Database...
Loading COCO dataset val2017 information....
Loading annotations into database...
	annotations COPIED.
	COMMITED.
Loading images into database...
	images COPIED.
	COMMITED.
Loading categories into database...
	categories COPIED.
	COMMITED.
Loading licenses into database...
	licenses COPIED.
	COMMITED.
Finished building database.


In [9]:
del build_trainset
del build_valset

In [12]:
qbd = QueryDatabase(**train_args)

Connecting to PostgreSQL server....
	Connected.


In [15]:
qbd.query_database('''
SELECT *
FROM images
;
''')

In [17]:
qbd.df_query

,id,file_name,coco_url,flickr_url,local_path,height,width,date_captured,license
0,391895,000000391895.jpg,http://images.cocodataset.org/train2017/000000...,http://farm9.staticflickr.com/8186/8119368305_...,../data/coco/train2017/000000391895.jpg,360,640,2013-11-14 11:18:45,3
1,522418,000000522418.jpg,http://images.cocodataset.org/train2017/000000...,http://farm1.staticflickr.com/1/127244861_ab0c...,../data/coco/train2017/000000522418.jpg,480,640,2013-11-14 11:38:44,4
2,184613,000000184613.jpg,http://images.cocodataset.org/train2017/000000...,http://farm3.staticflickr.com/2169/2118578392_...,../data/coco/train2017/000000184613.jpg,336,500,2013-11-14 12:36:29,3
3,318219,000000318219.jpg,http://images.cocodataset.org/train2017/000000...,http://farm5.staticflickr.com/4125/5094763076_...,../data/coco/train2017/000000318219.jpg,640,556,2013-11-14 13:02:53,3
4,554625,000000554625.jpg,http://images.cocodataset.org/train2017/000000...,http://farm5.staticflickr.com/4086/5094162993_...,../data/coco/train2017/000000554625.jpg,640,426,2013-11-14 16:03:19,3
5,574769,000000574769.jpg,http://images.cocodataset.org/train2017/000000...,http://farm8.staticflickr.com/7010/6728227647_...,../data/coco/train2017/000000574769.jpg,640,480,2013-11-14 17:07:59,3
6,60623,000000060623.jpg,http://images.cocodataset.org/train2017/000000...,http://farm7.staticflickr.com/6080/6113512699_...,../data/coco/train2017/000000060623.jpg,427,640,2013-11-14 17:24:15,4
7,309022,000000309022.jpg,http://images.cocodataset.org/train2017/000000...,http://farm4.staticflickr.com/3790/10167396295...,../data/coco/train2017/000000309022.jpg,480,640,2013-11-14 17:28:23,2
8,5802,000000005802.jpg,http://images.cocodataset.org/train2017/000000...,http://farm4.staticflickr.com/3810/9614287841_...,../data/coco/train2017/000000005802.jpg,479,640,2013-11-14 17:28:25,2
9,222564,000000222564.jpg,http://images.cocodataset.org/train2017/000000...,http://farm8.staticflickr.com/7390/10166966765...,../data/coco/train2017/000000222564.jpg,480,640,2013-11-14 17:30:34,2


### Create view for street context images

In [15]:
views = '''
-- Create quick view tables for street-context annotations and images


BEGIN;

DROP VIEW IF EXISTS street_images;
DROP VIEW IF EXISTS street_annotations;


CREATE VIEW street_annotations AS
    (SELECT
      an.image_id as image_id,
      cat.name as category,
      an.id as annotation_id,
      an.bbox as bbox
    FROM
      annotations as an
    JOIN
        categories as cat
      ON
        an.category_id = cat.id
    AND
        cat.supercategory
      IN ('vehicle', 'outdoor')
    AND
        cat.name
      NOT IN ('airplane', 'train', 'boat'))
;


CREATE VIEW street_images AS (
    SELECT
        DISTINCT image_id,
        img.file_name as file_name,
        img.coco_url as coco_url,
        img.local_path as local_path
    FROM
        street_annotations
    JOIN
            images as img
        ON
            image_id = img.id
);


COMMIT;

'''

In [16]:
qbd.connxn.rollback()
qbd.query_database(views)

ProgrammingError: no results to fetch

In [33]:
qbd.query_database('''
SELECT 
    image_id, 
    category,
    local_path
FROM
    street_images
''')
qbd.df_query.head()

,image_id,category,local_path
0,64,car,../data/coco/train2017/000000000064.jpg
1,64,stop sign,../data/coco/train2017/000000000064.jpg
2,64,truck,../data/coco/train2017/000000000064.jpg
3,71,car,../data/coco/train2017/000000000071.jpg
4,71,car,../data/coco/train2017/000000000071.jpg


### load data

In [2]:
%load_ext autoreload
%autoreload 1
%aimport trafficlight_data

In [7]:
from trafficlight_data import load_binary_train
X, y = load_binary_train(**train_args)

Connecting to PostgreSQL server....
	Connected.


In [8]:
X.head()

,category,bbox,file_name,coco_url,local_path
image_id,,,,,
64,car,"[b'[', b' ', b'5', b'2', b'.', b'3', b'6', b' ...",000000000064.jpg,http://images.cocodataset.org/train2017/000000...,../data/coco/train2017/000000000064.jpg
64,stop sign,"[b'[', b'1', b'1', b'5', b'.', b'8', b'4', b' ...",000000000064.jpg,http://images.cocodataset.org/train2017/000000...,../data/coco/train2017/000000000064.jpg
64,truck,"[b'[', b' ', b'5', b'2', b'.', b'6', b'4', b' ...",000000000064.jpg,http://images.cocodataset.org/train2017/000000...,../data/coco/train2017/000000000064.jpg
71,car,"[b'[', b'3', b'4', b'8', b'.', b'7', b'9', b' ...",000000000071.jpg,http://images.cocodataset.org/train2017/000000...,../data/coco/train2017/000000000071.jpg
71,car,"[b'[', b'3', b'6', b'6', b'.', b'6', b'9', b' ...",000000000071.jpg,http://images.cocodataset.org/train2017/000000...,../data/coco/train2017/000000000071.jpg


In [11]:
len(y), y.sum()

(26892, 4139)